## 네이버 뉴스 반응 수집

- 네이버뉴스 링크의 도메인에 따라 반응 수집 코드를 다르게 작성해야 함
- 네이버뉴스 링크의 도메인에 따라 반응 세부 항목이 서로 다르므로 행 방향 결합이 어려움
  - `https://n.news`로 시작하는 뉴스 반응은 '쏠쏠정보', '흥미진진', '공감백배', '분석탁월', '후속강추' 등 5개
  - `https://m.sports`로 시작하는 뉴스 반응은 '좋아요', '슬퍼요', '화나요', '팬이에요', '후속기사 원해요' 등 5개
  - `https://m.entertain`으로 시작하는 뉴스 반응은 '좋아요', '응원해요', '축하해요', '기대해요', '놀랐어요', '슬퍼요' 등 6개

### 데이터 준비

In [ ]:
# 관련 라이브러리를 호출합니다.
import os
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as BTS
import json
import re

In [ ]:
# 현재 작업 경로를 확인합니다.
os.getcwd()

In [ ]:
# data 폴더로 작업 경로를 변경합니다.
os.chdir(path = '../data')

In [ ]:
# 현재 작업 경로에 있는 폴더명과 파일명을 확인합니다.
sorted(os.listdir())

In [ ]:
# pkl 파일을 읽고 newsLinks를 생성합니다.
newsLinks = pd.read_pickle(filepath_or_buffer = 'Naver_News_Link.pkl')

In [ ]:
# newsLinks의 처음 5행을 확인합니다.
newsLinks.head()

In [ ]:
# newsLinks에서 Domain의 도수를 확인합니다.
newsLinks.str.extract(pat = '(https://.+?(?=/article))').value_counts()

### 뉴스 반응 수집 함수 생성

In [ ]:
# Domain에 따라 뉴스 반응을 수집하는 함수를 생성합니다.
def NaverNewsReact(link):
    
    # newsLinks에서 cid(company id)와 aid(article id)를 추출합니다.
    cid = re.findall(pattern = r'(?<=article/)(\d+)', string = link)[0]
    aid = re.findall(pattern = r'(?<=\d/)(\d+$)', string = link)[0]

    # 요청 헤더를 설정합니다.
    # [참고] referer에 네이버뉴스 링크를 지정합니다.
    headers = {
        'content-type': 'application/javascript;charset=UTF-8', 
        'referer': link, 
        'user-agent': 'Mozilla/5.0'
    }

    # 요청 URL과 쿼리 문자열을 설정합니다.
    if link.startswith('https://n.news'):
        url = 'https://news.like.naver.com/v1/search/contents'
        query = {
            'suppress_response_codes': 'true',
            'q': f'NEWS[ne_{cid}_{aid}]',
            'isDuplication': 'false',
            'cssIds': 'MULTI_MOBILE,NEWS_MOBILE'
        }
    elif link.startswith('https://m.sports'):
        url = 'https://apis.naver.com/sports/like/v1/search/contents'
        query = {
            'q': f'SPORTS[ne_{cid}_{aid}]',
            '_method': 'GET',
            'suppress_response_codes': 'true'
        }
    elif link.startswith('https://m.entertain'):
        url = 'https://news.like.naver.com/v1/search/contents'
        query = {
            'q': f'ENTERTAIN[ne_{cid}_{aid}]',
            '_method': 'GET',
            'suppress_response_codes': 'true'
        }

    # HTTP 요청을 실행합니다.
    res = requests.get(url = url, params = query, headers = headers)

    # JSON 형태의 문자열을 딕셔너리로 변환합니다.
    dat = json.loads(s = res.text)

    # 뉴스 반응 개수를 데이터프레임으로 변환합니다.
    reactCnt = pd.DataFrame(data = dat['contents'][0]['reactions'])
    
    # reactCnt의 첫 번째 열로 'nlink'를 추가합니다.
    reactCnt.insert(loc = 0, column = 'nlink', value = link)
    
    # reactCnt(Long Type)를 Wide Type으로 변환하고 reactCnt에 재할당합니다.
    reactCnt = reactCnt.pivot(index = 'nlink', columns = 'reactionType', values = 'count')
    
    # reactCnt의 행이름 초기화하고 reactCnt에 재할당합니다.
    reactCnt = reactCnt.reset_index()
    
    # reactCnt에서 열이름 네임을 삭제합니다.
    reactCnt.columns.name = None
    
    # reactCnt를 반환합니다.
    return reactCnt

In [ ]:
# 첫 번째 네이버뉴스 링크를 출력합니다.
print(newsLinks.iloc[0])

In [ ]:
# 첫 번째 네이버뉴스 링크로 반응 개수를 수집합니다.
NaverNewsReact(link = newsLinks.iloc[0])

## End of Document